In [1]:
#%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Matlab-like way to work with matrices
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
import scipy

# R-like way to work with dataframes
import pandas as pd

# Equivalent of brainstorm, EEGLAB, ...
import mne
from mne.io import read_raw_eeglab
from mne.viz import plot_topomap
from mne.channels import make_eeg_layout
from mne.preprocessing import compute_proj_eog
from mne.viz import circular_layout, plot_connectivity_circle
from mne.connectivity import spectral_connectivity

# Plotting libraries
import matplotlib.pylab as plt   # <= Similar to Matlab
import seaborn as sns            # <= Easy, nice-looking plotting  

from eegip.utils import loadmat # load_montage, 
from eegip.preprocessing import preprocess, channel_rejection, add_bad_segment_annot, \
    get_events, get_epochs, remove_rejected_ica_components, get_evoked
from eegip.dataset import DatasetMng
from eegip.path import get_path
from eegip import Analysis
from eegip.sources import SourcesBIDS

from sklearn.decomposition import PCA, FastICA
from mne.preprocessing import ICA

from warnings import warn
#from eegip.config import eegip_config
#from eegip.plotting import plot_mean_and_CI

# Others...
import xarray as xr
from pathlib import Path
import re
import os
from glob import glob

/home/christian/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
dataset_name="eegip_london"
config =  {"atlas":{"vol_labels":[]}}

template_check_path = Path("/home/christian/template_check/")
dataset_mng = DatasetMng(config=[config], dataset_name=dataset_name)
recording_names = sorted([name for name in dataset_mng.dataset.recordings if "m12" not in name])

# Compute scalp stats

In [5]:
for recording_name in tqdm(recording_names):
    recording = dataset_mng.dataset.recordings[recording_name]

    try:
        recording.get_artifact("events", recompute=True)                
        recording.get_artifact("epochs", recompute=True)
    except FileNotFoundError as e:
        continue

    data = recording.epochs.get_data()
    
    data = xr.DataArray(data, dims=('epochs', 'channels', 'times'),
                     coords={'epochs': np.arange(data.shape[0]), 
                             'channels': recording.epochs.ch_names,
                             'times': recording.epochs.times})

    data -= data.mean("times")  # De-meaning time series
    
    inv_event_id = {val:key for key, val in recording.epochs.event_id.items()}
    event_strs = np.array([inv_event_id[id_] for id_ in recording.epochs.events[:, 2]])

    recording.flush()
        
    for event_str in np.unique(event_strs):
        epochs_no = np.where(event_strs == event_str)[0] 

        file_path = template_check_path / "{}_{}_{}_mean.netcdf".format(recording_name, event_str, "scalp")
        data.sel(epochs=epochs_no).mean("epochs").to_netcdf(file_path, mode= 'w')

        file_path = template_check_path / "{}_{}_{}_std.netcdf".format(recording_name, event_str, "scalp")
        data.sel(epochs=epochs_no).std("epochs").to_netcdf(file_path, mode= 'w')

Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/eeg/sub-s601_ses-m06_eeg.fif...
    Range : 0 ... 81499 =      0.000 ...   162.998 secs
Ready.
116 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 116 events and 500 original time points ...
61 bad epochs dropped
Overwriting existing file.


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/eeg/sub-s601_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/eeg/sub-s602_ses-m06_eeg.fif...
    Range : 0 ... 240999 =      0.000 ...   481.998 secs
Ready.
374 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 374 events and 500 original time points ...


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/eeg/sub-s602_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


146 bad epochs dropped
Overwriting existing file.
Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s603/ses-m06/eeg/sub-s603_ses-m06_eeg.fif...
    Range : 0 ... 390999 =      0.000 ...   781.998 secs
Ready.
581 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 581 events and 500 original time points ...


...

    Range : 0 ... 287499 =      0.000 ...   574.998 secs
Ready.
440 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 440 events and 500 original time points ...
159 bad epochs dropped
Overwriting existing file.
Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s749/ses-m06/eeg/sub-s749_ses-m06_eeg.fif...


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s749/ses-m06/eeg/sub-s749_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


    Range : 0 ... 254499 =      0.000 ...   508.998 secs
Ready.
419 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 419 events and 500 original time points ...
138 bad epochs dropped
Overwriting existing file.
Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s750/ses-m06/eeg/sub-s750_ses-m06_eeg.fif...


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s750/ses-m06/eeg/sub-s750_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


    Range : 0 ... 228499 =      0.000 ...   456.998 secs
Ready.
374 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 374 events and 500 original time points ...
95 bad epochs dropped
Overwriting existing file.



# Compute ICA stats

In [6]:
template = "ica"
for recording_name in tqdm(recording_names):            
    recording = dataset_mng.dataset.recordings[recording_name]

    try:
        recording.get_artifact("events", recompute=True)                
        recording.get_artifact("epochs", recompute=True)
    except FileNotFoundError as e:
        continue

    recording.montage = mne.channels.make_standard_montage("GSN-HydroCel-129")

    inv_event_id = {val:key for key, val in recording.epochs.event_id.items()}
    event_strs = np.array([inv_event_id[id_] for id_ in recording.epochs.events[:, 2]])

    for event_str in np.unique(event_strs):
        ica = ICA(method='fastica', random_state=324234)
        ica = ica.fit(recording.epochs[event_str], verbose=False)
        ica_comp = ica.get_components()

        data = recording.epochs[event_str].get_data(picks="eeg").transpose([0, 2, 1]) @ ica_comp 
        recording.flush()        
        
        data = xr.DataArray(data, dims=('epochs', 'times', 'components'),
                         coords={'epochs': np.arange(data.shape[0]), 
                                 'components': np.arange(data.shape[2]),
                                 'times': recording.epochs.times})

        data -= data.mean("times")  # De-meaning time series

        file_path = template_check_path / "{}_{}_{}_mean.netcdf".format(recording_name, event_str, template)
        data.mean("epochs").to_netcdf(file_path, mode= 'w')

        file_path = template_check_path / "{}_{}_{}_std.netcdf".format(recording_name, event_str, template)
        data.std("epochs").to_netcdf(file_path, mode= 'w')

    recording.flush()

Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/eeg/sub-s601_ses-m06_eeg.fif...
    Range : 0 ... 81499 =      0.000 ...   162.998 secs
Ready.
116 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 116 events and 500 original time points ...
61 bad epochs dropped
Overwriting existing file.


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/eeg/sub-s601_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/mod/sub-s601_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
55 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/mod/sub-s601_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
55 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/mod/sub-s601_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0

/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/eeg/sub-s602_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


146 bad epochs dropped
Overwriting existing file.
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/mod/sub-s602_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
228 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/mod/sub-s602_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
228 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/mod/sub-s602_ses-m06_epo.fif ...
    Found the data of interest

...

159 bad epochs dropped
Overwriting existing file.
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s748/ses-m06/mod/sub-s748_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
281 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s748/ses-m06/mod/sub-s748_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
281 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s748/ses-m06/mod/sub-s748_ses-m06_epo.fif ...
    Found the data of interest

/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s749/ses-m06/eeg/sub-s749_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


138 bad epochs dropped
Overwriting existing file.
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s749/ses-m06/mod/sub-s749_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
281 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s749/ses-m06/mod/sub-s749_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
281 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s749/ses-m06/mod/sub-s749_ses-m06_epo.fif ...
    Found the data of interest

/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s750/ses-m06/eeg/sub-s750_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


95 bad epochs dropped
Overwriting existing file.
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s750/ses-m06/mod/sub-s750_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
279 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s750/ses-m06/mod/sub-s750_ses-m06_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     798.00 ms
        0 CTF compensation matrices available
279 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s750/ses-m06/mod/sub-s750_ses-m06_epo.fif ...
    Found the data of interest:

# Compute fsaverage stats

In [4]:
subject_dir = Path(os.environ["SUBJECTS_DIR"])

recompute_smodel = False
recompute_fmodel = False
recompute_sources = False
model_type="average"
inv_method="dSPM"
snr=1.0
lambda2=None

template = "fsaverage"
fs_subject = template
derivative_name="template_validation_" + template
for recording_name in tqdm(recording_names):    
    recording = dataset_mng.dataset.recordings[recording_name]

    try:
        recording.get_artifact("events", recompute=True)                
        recording.get_artifact("epochs", recompute=True)
    except FileNotFoundError as e:
        continue

    recording.montage = mne.channels.make_standard_montage("GSN-HydroCel-129")

    smodel = recording.subject.add_smodel(name=fs_subject, fs_subject=fs_subject,
                                          type_=model_type, exists="return")
    if recompute_smodel:
        smodel.recompute()

    print(fs_subject)
    fmodel = recording.add_fmodel(smodel, name=fs_subject, exists="return")
    #if fname_trans is not None:
    #    fmodel.fname_trans = fname_trans
    if recompute_fmodel:
        try:
            fmodel.recompute()
        except FileNotFoundError as e:
            print(e)
            print("Skipping {}. It seems not to be available in lossless qcr version.".format(recording_name))
            continue

    if dataset_mng.dataset_config["is_bids"]:
        sources = SourcesBIDS(fmodel, snr=snr, lambda2=lambda2, method=inv_method,
                              derivative_name=derivative_name)
    else:
        sources = Sources(fmodel, snr=snr, lambda2=lambda2, method=inv_method)

    if recompute_sources:            
        sources.get_data(recompute=True, type_="all")


    # Save label stats
    sources.compute_labels(recompute=recompute_sources, no_epochs=None)

    data = sources.get_data(type_="labels")            


    data -= data.mean("times")  # De-meaning time series

    inv_event_id = {val:key for key, val in recording.epochs.event_id.items()}
    event_strs = np.array([inv_event_id[id_] for id_ in recording.epochs.events[:, 2]])    
    
    recording.flush()
    fmodel.flush()
    sources.flush()

    for event_str in np.unique(event_strs):
        epochs_no = np.where(event_strs == event_str)[0] 

        file_path = template_check_path / "{}_{}_{}_mean.netcdf".format(recording_name, event_str, template)
        data.sel(epochs=epochs_no).mean("epochs").to_netcdf(file_path, mode= 'w')

        file_path = template_check_path / "{}_{}_{}_std.netcdf".format(recording_name, event_str, template)
        data.sel(epochs=epochs_no).std("epochs").to_netcdf(file_path, mode= 'w')

    # Removing full source from the hard drive for space reasons 
    sources.delete_sources()
    
    # No need to recompute the smodel for every recordings since it does not depend on the 
    # recording
    recompute_smodel = False         


Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/eeg/sub-s601_ses-m06_eeg.fif...
    Range : 0 ... 81499 =      0.000 ...   162.998 secs
Ready.
116 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 116 events and 500 original time points ...
61 bad epochs dropped
Overwriting existing file.


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s601/ses-m06/eeg/sub-s601_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


fsaverage
Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/eeg/sub-s602_ses-m06_eeg.fif...


/home/christian/synchedout/eegip/eegip/recording.py:209: RuntimeWarning: This filename (/home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s602/ses-m06/eeg/sub-s602_ses-m06_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  self._preprocessed_raw = mne.io.read_raw_fif(self.get_path("preprocessed_raw"), verbose=self.verbose)


    Range : 0 ... 240999 =      0.000 ...   481.998 secs
Ready.
374 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 374 events and 500 original time points ...
146 bad epochs dropped
Overwriting existing file.
fsaverage
Opening raw data file /home/christian/synchedout/eegip/eegip/tests/test_artifacts/derivatives/eegip_london_preprocessed/sub-s603/ses-m06/eeg/sub-s603_ses-m06_eeg.fif...


...

Processing epoch : 278 / 279
combining the current components...


278it [00:40,  5.48it/s]

Processing epoch : 279 / 279
combining the current components...


279it [00:40,  6.82it/s]


Applying sign flip for label grouping.

